Let's import all needed packages.

In [ ]:
import perceval as pcvl
import numpy as np

In [ ]:
from perceval.components.unitary_components import PS, BS, PERM
import numpy as np

## Use the symbolic skin for display
from perceval.rendering.circuit import DisplayConfig, SymbSkin
DisplayConfig.select_skin(SymbSkin)

Defining a state:

In [ ]:
state1 = pcvl.BasicState('|0,2,0,1>')

Another way to do it:

In [ ]:
state2 = pcvl.BasicState([0, 2, 0, 1])

Check that the states are indeed the same:

In [ ]:
state1 == state2

Let's define a phase shifter:

In [ ]:
phase_shifter = PS(phi = np.pi)

In [ ]:
pcvl.pdisplay(phase_shifter.definition())

In [ ]:
pcvl.pdisplay(phase_shifter)

Now a beam splitter:

In [ ]:
beam_splitter = BS()

In [ ]:
pcvl.pdisplay(beam_splitter.definition())

In [ ]:
pcvl.pdisplay(beam_splitter)

In [ ]:
pcvl.pdisplay(BS().compute_unitary())

Note that you can define the phases directly numerically, or define them as parameters with a name that can be set to a numerical value later one in your code.

In [ ]:
phase_shifter = PS(phi = np.pi/2)
pcvl.pdisplay(phase_shifter)

In [ ]:
phi_angle = pcvl.P('phi_angle')
phase_shifter = PS(phi = phi_angle)
pcvl.pdisplay(phase_shifter)

In [ ]:
phi_angle.set_value(1)
pcvl.pdisplay(phase_shifter)

Now let us create a small circuit with these components:

In [ ]:
circuit = pcvl.Circuit(3)  # Create a 3 mode circuit

In [ ]:
circuit.add(0, BS())

In [ ]:
pcvl.pdisplay(circuit)

In [ ]:
circuit.add(0, PS(phi=np.pi/2)).add(1, PS(phi=pcvl.P('phi'))).add(1, BS())

In [ ]:
pcvl.pdisplay(circuit)

In [ ]:
pcvl.pdisplay(circuit.U)

If we want to simulate this circuit we need to define a processor with a backend. 

In [ ]:
processor = pcvl.Processor("SLOS", circuit)

Let us add an input state:

In [ ]:
processor.with_input(pcvl.BasicState([1,1,1]))

Different algorithms can be chosen, but here let's sample from the circuit as we would do in an actual experiment:

In [ ]:
sampler = pcvl.algorithm.Sampler(processor)

sample_count = sampler.sample_count(1000)
print(sample_count['results'])

How can we fix this?

Alternative: getting the probability table with Analyzer.

In [ ]:
processor_naive = pcvl.Processor("Naive", circuit)
analyzer = pcvl.algorithm.Analyzer(processor_naive, [pcvl.BasicState([1,1,1])], '*')
pcvl.pdisplay(analyzer)

Check that the values are the same?

Instead of manually defining the components of a circuit, we can also define a unitary and Perceval will generate the corresponding circuit:

In [ ]:
circuit_matrix = pcvl.Unitary(pcvl.Matrix.random_unitary(3))

In [ ]:
processor_unitary = pcvl.Processor("SLOS", circuit_matrix)
processor_unitary.with_input(pcvl.BasicState([1,1,1]))

In [ ]:
sampler = pcvl.algorithm.Sampler(processor_unitary)

sample_count = sampler.sample_count(1000)
print(sample_count['results'])